<a href="https://colab.research.google.com/github/samadpls/TensorFlow-Model-Exploration/blob/main/Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Building an Iris Dataset Model: Classification and DNN Architecture

**created by [samadpls](https://github.com/samadpls)**



In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals

import pandas as pd

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


In [3]:
CSV_COLUMN_NAME = ['SepalLength', 'SepalWidth', 'PetalLength',
                   'PetalWidth', 'Species'
                ]
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
train_path = tf.keras.utils.get_file (
    'iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file(
    'iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train = pd.read_csv(train_path, names=CSV_COLUMN_NAME, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAME, header=0)

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [7]:
train.shape

(120, 4)

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
  # converting input to dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [9]:
feature_columns = []
for key  in train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key = key))
print(feature_columns)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [10]:
# estimator has bunch of pre made model so we using Deep Neural Network (DNN)
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    hidden_units = [30,10], # 30 for 1st Hidden layer and 10 for 2nd one (randomly decided)
    n_classes = 3
)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


In [15]:
classifier.train(
    input_fn = lambda : input_fn(train, train_y, training=True), # it need function in parameter and we didnt embeded input_fn inside a fn so need to use lambda
    steps = 5000) # we go dataset 5000 until we hit 5000, similar to epochs


In [16]:
eval_result = classifier.evaluate(
     input_fn = lambda : input_fn(test, test_y, training=False)
)
print(eval_result['accuracy']) # 93% great :)

0.93333334


In [17]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [18]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth':  [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth' : [0.5, 1.5, 2.1]
}

predictions = classifier.predict(input_fn = lambda : input_fn(predict_x))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  print(SPECIES[class_id],"==>",100*probability)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


Setosa ==> 85.38439869880676
Versicolor ==> 53.99140119552612
Virginica ==> 67.21073985099792
